In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

2023-05-12 16:12:14.831620: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv("training.csv")

In [3]:
df

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3
...,...,...
17995,im having ssa examination tomorrow in the morn...,0
17996,i constantly worry about their fight against n...,1
17997,i feel its important to share this info for th...,1
17998,i truly feel that if you are passionate enough...,1


In [4]:
train_texts, test_texts, train_labels, test_labels = train_test_split(df["text"], df["label"], test_size=0.2)

In [5]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_texts)

In [6]:
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

In [7]:
maxlen = 50
train_padded = pad_sequences(train_sequences, maxlen=maxlen, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=maxlen, padding='post', truncating='post')

In [9]:
num_classes = 6
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=50, input_length=maxlen),
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=num_classes, activation='softmax')
])

In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
model.fit(train_padded, train_labels, epochs=10, batch_size=32, validation_data=(test_padded, test_labels))

Epoch 1/10
450/450 [==============================] - 5s 8ms/step - loss: 1.5051 - accuracy: 0.3410 - val_loss: 1.2162 - val_accuracy: 0.5133
Epoch 2/10
450/450 [==============================] - 4s 8ms/step - loss: 1.1060 - accuracy: 0.5730 - val_loss: 0.8468 - val_accuracy: 0.6714
Epoch 3/10
450/450 [==============================] - 4s 8ms/step - loss: 0.7998 - accuracy: 0.6947 - val_loss: 0.7150 - val_accuracy: 0.7072
Epoch 4/10
450/450 [==============================] - 4s 8ms/step - loss: 0.6701 - accuracy: 0.7302 - val_loss: 0.6094 - val_accuracy: 0.7703
Epoch 5/10
450/450 [==============================] - 4s 8ms/step - loss: 0.5392 - accuracy: 0.7780 - val_loss: 0.5829 - val_accuracy: 0.8089
Epoch 6/10
450/450 [==============================] - 3s 8ms/step - loss: 0.4620 - accuracy: 0.8303 - val_loss: 0.5069 - val_accuracy: 0.8392
Epoch 7/10
450/450 [==============================] - 3s 8ms/step - loss: 0.3750 - accuracy: 0.8714 - val_loss: 0.4518 - val_accuracy: 0.8683
Epoch 

In [13]:
test_loss, test_accuracy = model.evaluate(test_padded, test_labels)
print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)

test_predictions = model.predict(test_padded)
test_pred_labels = np.argmax(test_predictions, axis=1)
test_true_labels = np.argmax(test_labels, axis=1)
print(classification_report(test_true_labels, test_pred_labels))

113/113 [==============================] - 0s 2ms/step - loss: 0.4858 - accuracy: 0.8656
Test loss: 0.48578688502311707
Test accuracy: 0.8655555844306946
113/113 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.93      0.94      0.93      1038
           1       0.95      0.88      0.91      1229
           2       0.75      0.86      0.80       284
           3       0.92      0.80      0.86       475
           4       0.73      0.83      0.78       455
           5       0.38      0.48      0.42       119

    accuracy                           0.87      3600
   macro avg       0.78      0.80      0.78      3600
weighted avg       0.88      0.87      0.87      3600

